In [2]:
from collections import defaultdict
from itertools import combinations

In [3]:
# Reading the data

# List of all transactions
transactions = []

# List of unique items
unique_items = set() 

# To improve performance in the later steps, instead of creating a frequency map, we create a map with item as the key and the list of all transaction indices where the item has been purchased
items_trxns = defaultdict(list)

with open('./data/transactions.txt', 'r') as f:
    transaction_id = 0
    for line in f:
        # Split by whitespace and convert to strings and then to int
        transaction = line.strip().split()
        transaction = [int(item) for item in transaction]
        transactions.append(transaction)

        # Set of unique items in the transactions and counting the frequency of each item
        for item in transaction:
            unique_items.add(item)
            items_trxns[item].append(transaction_id)

        transaction_id += 1

In [4]:
# View our frequency map
for item, trxn_list in items_trxns.items():
    print(f"Item: {item} \nTransactions: {trxn_list}")
    break

Item: 25 
Transactions: [0, 17, 171, 307, 381, 399, 636, 657, 674, 780, 827, 848, 978, 1072, 1257, 1319, 1325, 1530, 1561, 1600, 1643, 1652, 1657, 1685, 1779, 1793, 1801, 1855, 1879, 1920, 1922, 1955, 2119, 2230, 2263, 2320, 2418, 2579, 2737, 3038, 3064, 3066, 3080, 3161, 3190, 3191, 3535, 3719, 3749, 3920, 4003, 4013, 4039, 4044, 4077, 4382, 4557, 4601, 4711, 4825, 4876, 4901, 5102, 5294, 5318, 5372, 5463, 5501, 5570, 5593, 5754, 5848, 5871, 5873, 6068, 6170, 6242, 6299, 6398, 6442, 6567, 6835, 6900, 7040, 7046, 7218, 7378, 7430, 7562, 7634, 7711, 7769, 7885, 8012, 8065, 8204, 8391, 8440, 8508, 8560, 8614, 8708, 8730, 8861, 8914, 9125, 9196, 9267, 9277, 9299, 9306, 9407, 9445, 9452, 9565, 9627, 9814, 9854, 9879, 9929, 9996]


View the first 3 transactions

In [5]:
# View the data
for i in range(3):
    print(transactions[i], end="\n")

[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834]
[39, 120, 124, 205, 401, 581, 704, 814, 825, 834]
[35, 249, 674, 712, 733, 759, 854, 950]


Number of unique items in the transactions dataset

In [6]:
len(unique_items) # Number of unique items in the transactions or C1

866

The item with the largest frequency, the most frequently purchased item in the dataset of transactions

In [7]:
maximum_freq_item = None
maximum_freq = float('-inf')
for item, freq in items_trxns.items():
    if len(freq) > maximum_freq:
        maximum_freq = len(freq)
        maximum_freq_item = item

print(maximum_freq_item, maximum_freq)

368 823


Calculating the minimum support count

In [8]:
MIN_SUPPORT = 0.01 # 1% 
MIN_SUPPORT_COUNT = int(MIN_SUPPORT * len(transactions))

# View the support count
print(MIN_SUPPORT_COUNT)

100


In [9]:
# Calculating the C1 itemset
c1_itemset = defaultdict(int) # Itemset: Support

for itemset, trxn_list in items_trxns.items():
    c1_itemset[itemset] = len(trxn_list)

for itemset, freq in c1_itemset.items():
    print(f"1-Itemset: {itemset}\nSupport: {freq}")
    break

1-Itemset: 25
Support: 121


We can also verify this by checking in our items-transactions map

In [10]:
len(items_trxns[25])

121

In [11]:
# Generating F1
f1_itemset = defaultdict(int)

for itemset, freq in c1_itemset.items():
    if freq >= MIN_SUPPORT_COUNT:
        f1_itemset[itemset] = freq


for itemset, freq in f1_itemset.items():
    print(f"1-Frequent Itemset: {itemset}\nSupport: {freq}")
    break

1-Frequent Itemset: 25
Support: 121


In [12]:
# Number of F1 itemsets
len(f1_itemset)

379

In [13]:
# Apriori Principle is already applied as we are only considering the 1-itemsets which are frequent
# Generating C2
f1_itemset_keys = list(f1_itemset.keys())
C2_itemset_keys = []

for i in range(len(f1_itemset_keys)):
    for j in range(i + 1, len(f1_itemset_keys)):
        C2_itemset_keys.append((f1_itemset_keys[i], f1_itemset_keys[j]))

print(C2_itemset_keys[0]) # Print one sample to check
print(len(C2_itemset_keys))

C2_itemset = defaultdict(int)

for itemset_key in C2_itemset_keys:
    a, b = itemset_key
    common_trxns = len(set(items_trxns[a]) & set(items_trxns[b]))
    C2_itemset[itemset_key] = common_trxns

# Generating F2
f2_itemset = defaultdict(int)


for itemset, freq in C2_itemset.items():
    if freq >= MIN_SUPPORT_COUNT:
        f2_itemset[frozenset(itemset)] = freq


print(f"Number of Frequent 2-itemsets: {len(f2_itemset)}")

(25, 52)
71631
Number of Frequent 2-itemsets: 11


In [14]:
# THey are only 11, print
for itemset, freq in f2_itemset.items():
    print(f"Itemset: {itemset}\tSupport: {freq}")

Itemset: frozenset({368, 682})	Support: 128
Itemset: frozenset({368, 829})	Support: 127
Itemset: frozenset({368, 692})	Support: 114
Itemset: frozenset({825, 39})	Support: 151
Itemset: frozenset({704, 825})	Support: 135
Itemset: frozenset({704, 39})	Support: 139
Itemset: frozenset({217, 283})	Support: 104
Itemset: frozenset({227, 390})	Support: 101
Itemset: frozenset({722, 390})	Support: 103
Itemset: frozenset({217, 346})	Support: 149
Itemset: frozenset({829, 789})	Support: 119


In [15]:
# Generating C3
f2_itemset_keys = list(f2_itemset.keys())
C3_itemset_keys_intermediate = set()

for f2_itemset_key in f2_itemset_keys:
    C3_itemset_keys_intermediate.update(f2_itemset_key)

C3_itemset_keys_intermediate = list(combinations(C3_itemset_keys_intermediate, 3)) # Create all possible combinations of the unique items

C3_itemset_keys = []
# Checking if a subset of any combination exists and has a minimum support count less than MIN_SUPPORT_COUNT
# Checking APriori principle
for C3_itemset_key in C3_itemset_keys_intermediate:
    subsets_of_2 = list(combinations(C3_itemset_key, 2))
    
    # Check for support of these subsets
    to_push = True
    for subset in subsets_of_2:
        if frozenset(subset) not in f2_itemset_keys:
            to_push = False

    if to_push:
        C3_itemset_keys.append(C3_itemset_key)

print(len(C3_itemset_keys)) # Print one example. 
            


1


In [18]:
# Calculate the support count of C3
C3_itemset = defaultdict(int)


for itemset_key in C3_itemset_keys:
    a, b, c = itemset_key
    common_trxns = len(set(items_trxns[a]) & set(items_trxns[b]) & set(items_trxns[c]))
    C3_itemset[frozenset(itemset_key)] = common_trxns

# Generate F3
f3_itemset = defaultdict(int)

for itemset, freq in C3_itemset.items():
    if freq >= MIN_SUPPORT_COUNT:
        f3_itemset[frozenset(itemset)] = freq

print(f"Number of Frequent 3-itemsets: {len(f3_itemset)}\n")
print("-"*20)
for itemset, freq in f3_itemset.items():
    print(f"3-Frequent Itemset: {itemset}\nSupport: {freq}")
    print("-"*20)

Number of Frequent 3-itemsets: 1

--------------------
3-Frequent Itemset: frozenset({704, 825, 39})
Support: 128
--------------------
